In [1]:
import cv2 as cv
import datetime
import sys

import numpy as np
import os
from numba import jit
import struct

In [2]:
np.iinfo(np.uint16)

iinfo(min=0, max=65535, dtype=uint16)

In [4]:
def convert_kitti_bin_to_xyz(binFilePath):
    size_float = 4
    list_xyz = []
    list_intensity = list()
    with open(binFilePath, "rb") as f:
        byte = f.read(size_float * 4)
        while byte:
            x, y, z, intensity = struct.unpack("ffff", byte)
            point = np.array([x, y, z])
            list_intensity.append(intensity)
            list_xyz.append(point)
            byte = f.read(size_float * 4)
    return list_xyz,list_intensity

In [5]:
@jit(nopython=True)
def fill_vals(img,range_img, height_img,intensity_img,v,u,z,range_xy,R,intensity):
    px_max = 65535
    for vv,uu,zz,xy,rr,ii in zip(v,u,z,range_xy,R,intensity):

        img[int(uu)][int(vv)] = np.round((rr/120)*px_max)
        range_img[int(uu)][int(vv)] = np.round((xy/120)*px_max)
        height_img[int(uu)][int(vv)] = np.round(((zz+15)/15)*px_max)
        intensity_img[int(uu)][int(vv)] = np.round(ii*px_max)

    return img,range_img, height_img,intensity_img
def convert_cloudPoints_to_img(xyz,intensity, rows,cols):
    x = np.asarray(xyz[:,0])
    y = np.asarray(xyz[:,1])
    z = np.asarray(xyz[:,2])
    R = np.sqrt(x*x+y*y+z*z)
    range_xy = np.sqrt(x*x+y*y)
    pitch =np.arcsin(z/R)
    yaw = np.arctan2(y,x)
    FOV_up = np.max(pitch)
    FOV_down = np.min(pitch)
    FOV = FOV_up - FOV_down
    u = rows*((FOV_up - pitch)/(FOV))
    v = cols*((yaw+np.pi)/(2*np.pi))
    v = np.floor(v)
    u = np.floor(u)
    img = np.zeros((rows+1, cols+1),dtype=np.uint16)
    range_img = img.copy()
    height_img = img.copy()
    intensity_img = img.copy()
    img,range_img, height_img,intensity_img = fill_vals(img, range_img, height_img,intensity_img,v,u,z,range_xy,R,intensity)
    img = cv.medianBlur(img,3)
    range_img = cv.medianBlur(range_img,3)
    height_img = cv.medianBlur(height_img,3)
    intensity_img = cv.medianBlur(intensity_img,3)

    return img,range_img,height_img,intensity_img

In [6]:

def kitti_sequence_ot_360images_sequence(dataset_path,sequence_number):
    poses_path = dataset_path + "/poses/"
    sequences_path = dataset_path + "/sequences/"
    sequences_dir_list =  sorted(os.listdir(sequences_path))
    seq_dir = sequences_dir_list[sequence_number]
    poses = None
    colib_data = None
    times_data = None
    if sequence_number<11:
        poses_file_list =  sorted(os.listdir(poses_path))
        pose_file = poses_file_list[sequence_number]

        poses = np.loadtxt(poses_path+pose_file, dtype=float)
        poses = poses.reshape( poses.shape[0], 3, 4)

        c_data = np.loadtxt(sequences_path+seq_dir+"/calib.txt", delimiter=' ', usecols=range(1,13), dtype=float)

        colib_data = [np.array(i).reshape(3,4) for i in c_data]

        colib_data=np.array(colib_data)
        Tr = colib_data[4]
        times_data = np.loadtxt(sequences_path+seq_dir+"/times.txt", dtype=float)



    list_img360 = list()
    list_range_img360 = list()
    list_height_img360 = list()
    list_intensity_img360 = list()
    list_img_bird_eye_view = list()

    velodyne_file_list = sorted(os.listdir(sequences_path+seq_dir+"/velodyne/"))
    print("read files from :"+ sequences_path+seq_dir+"/velodyne:")

    for i in range(len(velodyne_file_list)):
        print(f'{i+1}/{len(velodyne_file_list)}', end="\r")
        sys.stdout.flush()
        bin_file = sequences_path+seq_dir+"/velodyne/"+velodyne_file_list[i]
        
        xyz_val ,intensity= convert_kitti_bin_to_xyz(bin_file)
        img,range_img,height_img,intensity_img = convert_cloudPoints_to_img(np.asarray(xyz_val), np.asarray(intensity), 64, 1024)
        img_bird_eye_view = point_cloud_2_birdseye(np.asarray(xyz_val) )
        list_img360.append(img)
        list_range_img360.append(range_img)
        list_height_img360.append(height_img)
        list_intensity_img360.append(intensity_img) 
        list_img_bird_eye_view.append(img_bird_eye_view)
        
 

    return poses, colib_data, times_data, list_img360,list_range_img360,list_height_img360,list_intensity_img360,list_img_bird_eye_view

In [7]:
def save_img_list(dataset_path,sequence_number,dataset_img_path, list_img360,list_range_img360,list_height_img360,list_intensity_img360,list_img_bird_eye_view):
    sequences_path = dataset_path + "/sequences/"
    sequences_dir_list =  sorted(os.listdir(sequences_path))
    seq_dir = sequences_dir_list[sequence_number]
    velodyne_file_list = sorted(os.listdir(sequences_path+seq_dir+"/velodyne/"))
    img360_dir = dataset_img_path+ "/sequences/"+seq_dir+"/img360"
    range_img360_dir = dataset_img_path+ "/sequences/"+seq_dir+"/range_img360"
    height_img360_dir = dataset_img_path+ "/sequences/"+seq_dir+"/height_img360"
    intensity_img360_dir = dataset_img_path+ "/sequences/"+seq_dir+"/intensity_img360"
    bird_eye_view_img360_dir = dataset_img_path+ "/sequences/"+seq_dir+"/bird_eye_view"
    try:
        os.makedirs(img360_dir)
        os.makedirs(range_img360_dir)
        os.makedirs(height_img360_dir)
        os.makedirs(intensity_img360_dir)
        os.makedirs(bird_eye_view_img360_dir)

    except FileExistsError:
        print("File already exists")
    print('write at sequences '+seq_dir+':')

    i=1
    for bin_file_name,img360,range_img360,height_img360,intensity_img360, img_bird_eye_view in zip(velodyne_file_list,list_img360,list_range_img360,list_height_img360,list_intensity_img360,list_img_bird_eye_view):
        print(f'{i}/{len(velodyne_file_list)}', end="\r")
        sys.stdout.flush()
        i=i+1
        img_file = img360_dir+'/'+bin_file_name[:-4]+'.png'
        range_file = range_img360_dir+'/'+bin_file_name[:-4]+'.png'
        height_file = height_img360_dir+'/'+bin_file_name[:-4]+'.png'
        intensity_file = intensity_img360_dir+'/'+bin_file_name[:-4]+'.png'
        bird_eye_view_file = bird_eye_view_img360_dir+'/'+bin_file_name[:-4]+'.png'
        cv.imwrite(img_file,img360)
        cv.imwrite(range_file,range_img360)
        cv.imwrite(height_file,height_img360)
        cv.imwrite(intensity_file,intensity_img360)
        cv.imwrite(bird_eye_view_file,img_bird_eye_view)

In [8]:
dataset_path = "/home/muhammed/ara/dataset"
dataset_img_path = '/home/muhammed/ara/dataset_img_v4'

In [9]:
for sequence_number in range(22):
    _, _, _, list_img360,list_range_img360,list_height_img360,list_intensity_img360,list_img_bird_eye_view = kitti_sequence_ot_360images_sequence(dataset_path,sequence_number)
    save_img_list(dataset_path,sequence_number,dataset_img_path, list_img360,list_range_img360,list_height_img360,list_intensity_img360,list_img_bird_eye_view)

read files from :/home/muhammed/ara/dataset/sequences/00/velodyne:
write at sequences 00:
read files from :/home/muhammed/ara/dataset/sequences/01/velodyne:


: 

: 